gpt2: decoder only model, so the encoder & the cross attention all missed

In [2]:
from dataclasses import dataclass
import math
import torch
import torch.nn as nn
from torch.nn import functional as F
import inspect
import tiktoken

In [3]:
# attempt to sutodetect the device, cpu, cuda and mps(Accelerated PyTorch training on Mac)
DEVICE = torch.device('mps' if torch.backends.mps.is_available() else ('cuda' if torch.cuda.is_available() else 'cpu'))
print(f"using device: {DEVICE}")

using device: cpu


In [ ]:
@dataclass
class GPTConfig:
    block_size: int = 1024      # max sequence length
    vocab_size: int = 50257     # number of tokens: 50,000 BPE tokens + 256 bytes tokens + 1 <|end of the text|> token
    n_layer: int = 12           # number of layers
    n_head: int = 12            # number of heads
    n_embed: int = 768          # embedding dimensions
    
    
class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embed % config.n_head == 0
        
        # key, query, value projection for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embed, 3 * config.n_embed)
        
        # output projection
        self.c_proj = nn.Linear(config.n_embed, config.n_embed)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        
        # regularization
        self.n_head = config.n_head
        self.n_embed = config.n_embed
        
        # not really a 'bias', more of a mask, but following the OPENAI/HF naming though
        self.register_buffer(
            'bias',
            torch.tril(
                torch.ones(config.block_size, config.block_size)
            ).view(1, 1, config.block_size, config.block_size)
        )

    def forward(self, x):
        B, T, C = x.size() # batch_size, sequence_length, embedding dimensionality (n_embed)
        # calculate query, key, value for all heads and move head forward to be the batch 
        # nh: number of heads; hs: head size, C: number of channels = nh * hs
        
        qkv = self.c_attn(x)
        q, k, v = qkv.split(self.n_embed, dim=2)
        
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        
        # attention (materials the large (T, T) matrix for all queries and keys)
        # attn = (q @ k.transpose(-2, -1)) * (1. / math.sqrt(k.size(-1)))
        # attn = attn.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
        # attn = F.softmax(attn, dim=-1)
        # y = attn @ v # (B, nh, T, T) * (B, nh, T, hs) -> (B. nh, T. hs)
        y = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        
        # output projection
        y = self.c_proj(y)
        
        return y 

class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        self.c_fc = nn.Linear(config.n_embed, 4 * config.n_embed)
        self.gelu = nn.GELU(approximate='tanh') # a smoother relu
        self.c_proj = nn.Linear(4 * config.n_embed, config.n_embed)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        
    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embed)
        self.attn = CausalSelfAttention(config)
        # attention is  a communication operation, 
                        # a aggregation function, 
                        # a polling function, 
                        # a weighted sum function, 
                        # a reduce operation
        self.ln_2 = nn.LayerNorm(config.n_embed)
        self.mlp = MLP(config)
    
    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x        


class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embed),  # token embedding
            wpe = nn.Embedding(config.block_size, config.n_embed),  # positional encoding
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embed)
        ))
        
        self.lm_head = nn.Linear(config.n_embed, config.vocab_size, bias=False)
        
        # weight sharing scheme
        self.transformer.wte.weight = self.lm_head.weight # for the same target

        # init params
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            std = 0.02
            if hasattr(module, "NANOGPT_SCALE_INIT"):
                std *= (2 * self.config.n_layer) ** -0.5
            torch.nn.init.normal_(module.weight, mean=0.0, std=std)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, idx, targets=None):
        # idx is of shape(B, T)
        B, T = idx.size()
        assert T <= self.config.block_size, f"Cannot forward sequence of length {T}, block size"
        
        # forward the token and position embeddings
        pos = torch.arange(0, T, dtype=torch.long, device=idx.device) # shape (T)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (T, n_embed)
        tok_emb = self.transformer.wte(idx) # token embedding of shape (B, T, n_embed)
        x = pos_emb + tok_emb
        
        # forward the blocks of the transformer
        for block in self.transformer.h:
            x = block(x)
            
        # forward the final layernorm and the classifier
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(
                logits.view(-1, logits.size(-1)), targets.view(-1)
            )
        return logits, loss
        
        
    @classmethod
    def from_pretrain(cls, model_type):
        '''loads pretrained GPT-2 model weights from huggingface'''
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print(f"loading weights from pretrained gpt: {model_type}")
        
        # n_layer, n_head, n_embed are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embed=768), # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embed=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embed=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=48, n_embed=1600), # 1558M params
        }[model_type]
        
        config_args['vocab_size'] = 50257   # always
        config_args['block_size'] = 1024    # always
        config = GPTConfig(**config_args)
        model = GPT(config)
        
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask
        
        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()
        
        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')]
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')]
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])
        
        return model
    
    def configure_optimizer(self, weight_decay, learning_rate, device):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        
        # create optim groups, any parameters that is 2D will be weight_decayed, otherwise no
        # i.e. all weight tensors in matmul + embeddings decay, while all biases and layernorms don't
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters") # ":,"添加千分位符号
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        
        # create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and 'cuda' in device
        print(f"using fused AdamW: {use_fused}")
        optimizer = torch.optim.AdamW(
            optim_groups, lr=learning_rate, betas=(0.9, 0.95), eps=1e-8, fused=use_fused
        )
        return optimizer
        
        
class DataloaderLite:
    def __init__(self, B, T, process_rank=0, num_processes=1):
        self.B = B
        self.T = T
        self.process_rank = process_rank
        self.num_processes = num_processes
        
        # an init load tokens from disk and store them in memory
        with open('./datasets/input.txt', "r") as f:
            text = f.read()
        
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f"loaded {len(self.tokens)} tokens")
        print(f"1 epoch = {len(self.tokens) // (B * T)} batches")
        
        # state
        self.current_position = self.B * self.T * self.process_rank
        
    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position: self.current_position + B * T + 1]
        x = buf[:-1].view(B, T) # inputs
        y = buf[1:].view(B, T)  # targets
        
        # advance the position in the tensor
        self.current_position += B * T * self.num_processes
        
        # if loading the next position would be out of bounds, reset
        if self.current_position + (B * T * self.num_processes + 1) > len(self.tokens):
            self.current_position = self.B * self.T * self.process_rank
        
        return x, y

In [5]:
num_return_sequences = 5
max_length = 30


# model = GPT.from_pretrain('gpt2')
# 随机初始化的GPT
model = GPT(GPTConfig())
model.eval()
model.to(DEVICE)
print("didn't crash yay!")

# prefix tokens
import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello, I'm a language model")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
x = tokens.to(DEVICE)

# generate right now x is (B, T) where B = 5, T = 8
torch.manual_seed(42)
torch.cuda.manual_seed(42)

while x.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        logits, _ = model(x) # (B, T, vocab_size)
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top k sampling of 50 (higgingface pipeline default)
        # top-k _probs here becomes (5, 50), topk-indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        ix = torch.multinomial(topk_probs, 1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        x = torch.cat((x, xcol), dim=1)
        
# print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)

didn't crash yay!
> Hello, I'm a language model MelASED McKin FTP HIV reconciliation Check Friedman nasty gamma therapist bid Yet OrnTraditional Training inhabENG Timothy Letterricannegative Orn
> Hello, I'm a language model Leaders imposingients Angels � Method exilefemale blindness reconciliation LH disguisedMAX [+ENG €ude blamed Timothy recoveroricalASEDOriginal
> Hello, I'm a language modelGAME ### disruptingnet InvestorIntelkers journal attained [+ comparisonsKay Using Lt fading Valhallasterdam Guilty fading Meleeanka weighted vap
> Hello, I'm a language modelacer Job Meyer Orn cricketSeg AirportYL OrnENGmAh compulsion La emph Pier Tour blinked Footballude BLMalshUk pul
> Hello, I'm a language modelimates repeated617 BOOK bike bike enrol Pier asideVict Mor Gw deputiesKay matcheditivesFI343 vil DDR unfold facynamic


In [6]:
# try the model
# train the model
# tiny Shakespeare Dataset, for debugging
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

# import tiktoken
# import torch


# get a data batch
# with open("./datasets/input.txt", "r") as f:
#     text = f.read()

# data = text[:1000] # first 1000 characters
# print(data[:100])

# enc = tiktoken.get_encoding('gpt2')
# tokens = enc.encode(data)
# # print(tokens[:24])

# B, T = 4, 32
# buf = torch.tensor(tokens[:B * T + 1])
# buf = buf.to(DEVICE)

# # x = buf.view(4, 6)
# x = buf[:-1].view(B, T)
# y = buf[1:].view(B, T) # targets
# print(x.shape)

In [15]:
import torch
import time

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

total_batch_size = 524_288 # 2 ** 19
B = 16      # micro batch size
T = 1024    # sequence length
assert total_batch_size % (B * T) == 0, "make sure total batch_size is divisible by B * T"
grad_accum_steps = total_batch_size // (B * T)
print(f"total desired batch_size: {total_batch_size}")
print(f"=> calculated gradient accumulation steps: {grad_accum_steps}")

train_loader = DataloaderLite(B=B, T=T)
# train_loader = DataloaderLite(B=4, T=24)
torch.set_float32_matmul_precision('high') # float32 -> tensorflow32(16bit)

# get logits
model = GPT(GPTConfig)
model.to(DEVICE)
# model = torch.compile(model)

max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_steps = 10
max_steps = 50
def get_lr(it):
    # 1. linear warm up for warmup_iter steps
    if it < warmup_steps:
        return max_lr * (it + 1) / warmup_steps
    # 2. if it > lr_decay_steps, return min learning rate
    if it > max_steps:
        return min_lr
    # 3. in between, use cousin decay down to min lr
    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # 1 -> 0
    return min_lr + coeff * (max_lr - min_lr)

# optimize!
# AdamW is a debug version of Adam
# optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8) 
optimizer = model.configure_optimizer(weight_decay=0.1, learning_rate=6e-4, device=str(DEVICE))
for step in range(max_steps):
    t0 = time.time()
    optimizer.zero_grad()
    for micro_step in range(grad_accum_steps):
        x, y = train_loader.next_batch()
        x, y = x.to(DEVICE), y.to(DEVICE)
        # logits, loss = model(x, y)
        with torch.autocast(device_type=str(DEVICE), dtype=torch.bfloat16):
            logits, loss = model(x, y)
        # we have to scale the loss to acocunt for gradient accumulation
        # because the gradients just add on each successive backward()
        # addition of gradients corresponds to a SUM of the objective, but 
        # instead of a SUM we want MEAN, Scale the loss here ao it comes out right
        loss = loss / grad_accum_steps
        loss.backward() # add up without optimizer.zero_grad(), but need average
        
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    # determine and set the lr for this iteration
    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    
    optimizer.step()
    if DEVICE == "cuda":
        torch.cuda.synchronize()
    
    t1 = time.time()
    dt = (t1 - t0) * 1000 # time difference in miliseconds
    tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
    print(f"step {step}, loss: {loss.item():.6f}, | norm: {norm:.4f} | lr: {lr:.4f} | dt: {dt:.2f}ms, {tokens_per_sec:4f} tokens per second")


loaded 338025 tokens
1 epoch = 3521 batches
num decayed parameter tensors: 50, with 124,318,464 parameters
num non-decayed parameter tensors: 98, with 121,344 parameters
using fused AdamW: False
step 0, loss: 10.919169, | norm: 52.7442 | lr: 0.0001 | dt: 46660.74ms, 2.057404 tokens per second
step 1, loss: 9.470251, | norm: 16.2002 | lr: 0.0001 | dt: 47082.21ms, 2.038987 tokens per second
step 2, loss: 9.682421, | norm: 9.8413 | lr: 0.0002 | dt: 51798.38ms, 1.853340 tokens per second
step 3, loss: 9.733609, | norm: 11.0809 | lr: 0.0002 | dt: 47109.27ms, 2.037816 tokens per second
step 4, loss: 9.228730, | norm: 5.1816 | lr: 0.0003 | dt: 47014.59ms, 2.041919 tokens per second
step 5, loss: 8.582575, | norm: 4.9198 | lr: 0.0004 | dt: 46467.06ms, 2.065980 tokens per second
step 6, loss: 8.739457, | norm: 3.5296 | lr: 0.0004 | dt: 46842.85ms, 2.049405 tokens per second
step 7, loss: 8.184897, | norm: 3.9794 | lr: 0.0005 | dt: 46617.50ms, 2.059313 tokens per second
step 8, loss: 8.814713, |

In [ ]:
# if multiple gpu
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
from torch.distributed import init_process_group, destroy_process_group
import os
# set up DDP (distributed data parallel)
# torchrun command sets to the env variables RANK, and WORLD_SIZE
ddp = int(os.environ.get('RANK', -1)) != -1 # is this ddp run?

if ddp:
    # use the DDP atm demands CUDA, we set the device appropriately according to rank
    assert torch.cuda.is_available(), "CUDA is required for DDP"
    init_process_group(backend='nccl')
    ddp_rank = int(os.environ('RANK'))
    ddp_local_rank = int(os.environ('LOCAL_RANK'))
    ddp_local_size = int(os.environ('LOCAL_SIZE'))
    DEVICE = f"cuda: {ddp_local_rank}"
    torch.cuda.set_device(DEVICE)
    master_process = ddp_rank == 0  # this process will do logging, checkpointintg etc.

else:
    # canilla, non-DDP run
    ddp_rank = 0
    ddp_local_rank = 0
    ddp_world_size = 1
    master_process = True
    
    # attempt to auto-detect device
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"using device: {DEVICE}")

torch.manual_seed(1337)
if torch.cuda.is_available():
    torch.cuda.manual_seed(1337)

total_batch_size = 524_288 # 2 ** 19
B = 16      # micro batch size
T = 1024    # sequence length
assert total_batch_size % (B * T * ddp_world_size) == 0, "make sure total batch_size is divisible by B * T * ddp_world_size"
grad_accum_steps = total_batch_size // (B * T * ddp_world_size)
if master_process:
    print(f"total desired batch_size: {total_batch_size}")
    print(f"=> calculated gradient accumulation steps: {grad_accum_steps}")


train_loader = DataloaderLite(B=B, T=T, process_rank=ddp_rank, num_processes=ddp_world_size)
# train_loader = DataloaderLite(B=4, T=24)
torch.set_float32_matmul_precision('high') # float32 -> tensorflow32(16bit)

# create model
model = GPT(GPTConfig)
model.to(DEVICE)
model = torch.compile(model)
if ddp:
    model = DDP(model, device_ids=[ddp_local_rank])
raw_model = model.module if ddp else model # always contains the RAW Unwrapped model


max_lr = 6e-4
min_lr = max_lr * 0.1
warmup_steps = 10
max_steps = 50

def get_lr(it):
    # 1. linear warm up for warmup_iter steps
    if it < warmup_steps:
        return max_lr * (it + 1) / warmup_steps
    # 2. if it > lr_decay_steps, return min learning rate
    if it > max_steps:
        return min_lr
    # 3. in between, use cousin decay down to min lr
    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # 1 -> 0
    return min_lr + coeff * (max_lr - min_lr)

# optimize!
# AdamW is a debug version of Adam
# optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, betas=(0.9, 0.95), eps=1e-8) 
optimizer = raw_model.configure_optimizer(weight_decay=0.1, learning_rate=6e-4, device=str(DEVICE))
for step in range(max_steps):
    t0 = time.time()
    optimizer.zero_grad()
    loss_accum = 0.0
    for micro_step in range(grad_accum_steps):
        x, y = train_loader.next_batch()
        x, y = x.to(DEVICE), y.to(DEVICE)
        # logits, loss = model(x, y)
        with torch.autocast(device_type=str(DEVICE), dtype=torch.bfloat16):
            logits, loss = model(x, y)
        # we have to scale the loss to acocunt for gradient accumulation
        # because the gradients just add on each successive backward()
        # addition of gradients corresponds to a SUM of the objective, but 
        # instead of a SUM we want MEAN, Scale the loss here ao it comes out right
        loss = loss / grad_accum_steps
        loss_accum += loss.detach()
        if ddp:
            model.require_backward_grad_sync = (micro_step == grad_accum_steps - 1)
        loss.backward() # add up without optimizer.zero_grad(), but need average
    if ddp:
        dist.all_reduce(loss_accum, op=dist.ReduceOp.AVG)
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    
    # determine and set the lr for this iteration
    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    
    optimizer.step()
    if DEVICE == "cuda":
        torch.cuda.synchronize()
    
    t1 = time.time()
    dt = (t1 - t0) * 1000 # time difference in miliseconds
    tokens_per_sec = (train_loader.B * train_loader.T) / (t1 - t0)
    if master_process:
        print(f"step {step}, loss: {loss_accum.item():.6f}, | norm: {norm:.4f} | lr: {lr:.4f} | dt: {dt:.2f}ms, {tokens_per_sec:4f} tokens per second")

    

using device: cpu
total desired batch_size: 524288
=> calculated gradient accumulation steps: 32
I'm gpu: 0


SystemExit: 0

/home/shenc/anaconda3/envs/gpt/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [19]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found
